In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import torch
from image_classification_simulation.data.office31_loader import Office31Loader
from image_classification_simulation.models.clustering_tools import show_grid_images
from image_classification_simulation.image_search import ImageSimilaritySearch
from image_classification_simulation.utils.visualization_utils import show_grid_images
from image_classification_simulation.models.clustering_tools import get_clustering_metrics
from image_classification_simulation.models.clustering_tools import (
    eval_clustering_performance, accuracy_per_class
)

In [2]:
hparams = {
    "num_workers": 2,
    'batch_size': 32,
    "image_size":224,
    "train_test_split":-1,
}
office_loader = Office31Loader(
    data_dir="../examples/data/domain_adaptation_images/amazon/images/",
    eval_dir="../examples/data/domain_adaptation_images/dslr/images/",
     hyper_params=hparams)
# office_loader.setup('fit')
# train_loader = office_loader.train_dataloader()
# val_loader = office_loader.val_dataloader()
# test_loader = office_loader.test_dataloader()
# # /network/projects/aia/img_classif_sim/vit/output/best_model
office_loader.setup('eval')
eval_loader = office_loader.eval_dataloader(shuffle=False)
office_loader.setup('infer')
train_loader = office_loader.train_dataloader(shuffle=False)

In [3]:

hparams_resnet = {
    "clustering_alg": "nn",
    "num_neighbors":20,
    "radius":0.5,
    "n_jobs":2,
    "loss": "CrossEntropyLoss",
    "batch_size": 100,
    "pretrained": True,
    "num_classes": 31,
    "path_to_model": "../examples/resnet/output/best_model/model.ckpt",
    "architecture": "resnet",
    "num_clusters": 31,
    "random_state": 0,
    "clustering_batch_size": 100,
    "size": 256,
    "reassignment_ratio": 0.05,
    "path_cluster_ids": "../debug/dataset_cluster_ids.csv",
}
hparams_vit = {
    # "clustering_alg": "MiniBatchKMeans",
    "clustering_alg": "nn",
    "num_neighbors":20,
    "radius":0.5,
    "n_jobs":2,
    "loss": "CrossEntropyLoss",
    "pretrained": True,
    "batch_size": 100,
    "num_classes": 31,
    "path_to_model": "/network/projects/aia/img_classif_sim/vit/output/best_model/model.ckpt",
    "architecture": "vit",
    "num_clusters": 100,
    "random_state": 0,
    "clustering_batch_size": 1024,
    "reassignment_ratio": 0.01,
    "init":'random',
    "path_cluster_ids": "../debug/dataset_cluster_ids.csv",
}
hparams_ae = {
    "clustering_alg": "MiniBatchKMeans",
    "loss": "CrossEntropyLoss",
    "pretrained": True,
    "batch_size": 100,
    "num_channels": 3,
    "num_classes": 31,
    "path_to_model": "/network/projects/aia/img_classif_sim/conv_ae/output/best_model/model.ckpt",
    "architecture": "conv_ae",
    "num_clusters": 32,
    "random_state": 0,
    "clustering_batch_size": 100,
    "reassignment_ratio": 0.05,
    "path_cluster_ids": "../debug/dataset_cluster_ids.csv",
}
hparams_cnn = {
        "clustering_alg": "nn",
        "num_neighbors":20,
        "radius":0.5,
        "n_jobs":2,
        "loss": "CrossEntropyLoss",
        "batch_size": 124,
        "num_channels": 3,
        "pretrained": True,
        "num_classes": 31,
        "img_size": 224,
        "path_to_model": "/network/projects/aia/img_classif_sim/classic_cnn/output/best_model/model.ckpt",
        "architecture": "classic-cnn",
        "num_clusters": 31,
        "random_state": 0,
        "clustering_batch_size": 124,
        "reassignment_ratio": 0.01,
        "path_cluster_ids": "../debug/dataset_cluster_ids.csv",
    }

archs = {
    "resnet": hparams_resnet,
    "vit": hparams_vit,
    # "ae": hparams_ae,
    "cnn":hparams_cnn
    }


In [6]:
labels_true = [label for image, label in office_loader.eval_set]
len(labels_true)


498

In [ ]:
labels_pred = image_search.predict(office_loader.eval_dataloader())
len(labels_pred)

The following metrics do not work when we are using nearest neighbors.

In [7]:
m = get_clustering_metrics(labels_true, labels_pred)
m

{'rand_score': 0.024261940161551716,
 'adjusted_rand_score': 0.024261940161551716,
 'mutual_info_score': 0.5971366373315987}

In [21]:
m

{'rand_score': 0.03455624029458484,
 'adjusted_rand_score': 0.03455624029458484,
 'mutual_info_score': 0.6542970012509423}

you can use either images from the evaluation or the training set

In [4]:
data_dir = "../examples/data/domain_adaptation_images/amazon/images/"

In [5]:
eval_dir = '../examples/data/domain_adaptation_images/dslr/images/'

In [ ]:
import os
import matplotlib.pyplot as plt

for arch in archs:
    image_search = ImageSimilaritySearch(archs[arch], office_loader)
    image_search.setup()
    for class_name in office_loader.dataset.class_to_idx:
        print(class_name)
        path = eval_dir+"{}/frame_0001.jpg".format(class_name)
        query_res = image_search.find_similar_images(path,None)
        fig,_ = show_grid_images(
            query_res['image_path'].tolist(),
            num_rows=5,
            num_cols=5,
            )
        fig.savefig('./results/'+arch+'/'+class_name+'.png',format='png')

In [16]:
image_search = ImageSimilaritySearch(archs['resnet'], office_loader)
image_search.setup()
class_ids = office_loader.labels
true_labels = [label for img, label in office_loader.eval_set]
total_score, res = eval_clustering_performance(
        class_ids,
        true_labels,
        image_search.clustering.find_neighbors,
        eval_loader,
        5
    )
class_acc = accuracy_per_class(true_labels, res)

In [23]:
for key,value in office_loader.dataset.class_to_idx.items():
    print(key,class_acc[value])

back_pack 0.8
bike 0.9238095238095239
bike_helmet 0.85
bookcase 0.6
bottle 0.85
calculator 0.8
desk_chair 0.8923076923076924
desk_lamp 0.4
desktop_computer 0.14666666666666667
file_cabinet 0.4533333333333333
headphones 0.5230769230769231
keyboard 0.52
laptop_computer 0.44166666666666665
letter_tray 0.5
mobile_phone 0.16129032258064516
monitor 0.35454545454545455
mouse 0.9166666666666666
mug 0.925
paper_notebook 0.74
pen 0.74
phone 0.6923076923076923
printer 0.4
projector 0.30434782608695654
punchers 0.2222222222222222
ring_binder 0.26
ruler 0.6571428571428571
scissors 0.9222222222222223
speaker 0.16153846153846155
stapler 0.2761904761904762
tape_dispenser 0.34545454545454546
trash_can 0.4266666666666667


In [27]:
image_search = ImageSimilaritySearch(archs['vit'], office_loader)
image_search.setup()
class_ids = office_loader.labels
true_labels = [label for img, label in office_loader.eval_set]
total_score, res = eval_clustering_performance(
        class_ids,
        true_labels,
        image_search.clustering.find_neighbors,
        eval_loader,
        5
    )
class_acc = accuracy_per_class(true_labels, res)

In [31]:
class_acc = accuracy_per_class(true_labels, res)
r = dict()
for key,value in office_loader.dataset.class_to_idx.items():
    r[key]=class_acc[value]

In [34]:
pd.DataFrame(data=r.values(),index=r.keys())

,0
back_pack,1.000000
bike,1.000000
bike_helmet,0.941667
bookcase,0.800000
bottle,0.125000
calculator,0.933333
desk_chair,1.000000
desk_lamp,0.842857
desktop_computer,0.813333
file_cabinet,0.946667
